**Notas para contenedor de docker:**

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

```
docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_openblas -p 8888:8888 -d palmoreck/jupyterlab_openblas:1.1.0
```

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

```
docker stop jupyterlab_openblas
```

**Al ejecutar el `run` anterior se descargará la imagen y posteriormente se instalará [OpenBLAS](https://github.com/xianyi/OpenBLAS) adaptado a sus sistemas. Esto tardará $10$ minutos aproximadamente.**

Documentación de la imagen de docker `palmoreck/jupyterlab_openblas:1.1.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/openblas).

---

Nota generada a partir de [liga](https://www.dropbox.com/s/jwu8lu4r14pb7ut/3.2.1.Sistemas_de_ecuaciones_lineales_eliminacion_Gaussiana_y_factorizacion_LU.pdf?dl=0), [liga2](https://www.dropbox.com/s/s4ch0ww1687pl76/3.2.2.Factorizaciones_matriciales_SVD_Cholesky_QR.pdf?dl=0)

# 3.2.Solución de sistemas de ecuaciones lineales y factorizaciones matriciales con OpenBLAS